<a href="https://colab.research.google.com/github/tharushaliyanagama/OralCancerEarlyDetection-DSGP/blob/Image-Validation/mobileNetV2_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/Proper_dataset/train"
test_dir = "/content/drive/MyDrive/Proper_dataset/val"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), class_mode='binary')


Found 2603 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 without top layers
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model.fit(train_data, validation_data=test_data, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.8617 - loss: 0.6074

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 1005s 12s/step - accuracy: 0.8622 - loss: 0.6040 - val_accuracy: 0.9539 - val_loss: 0.1119
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9625 - loss: 0.1042 - val_accuracy: 0.9631 - val_loss: 0.1042
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.9794 - loss: 0.0637 - val_accuracy: 0.9647 - val_loss: 0.1045
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9830 - loss: 0.0477 - val_accuracy: 0.9677 - val_loss: 0.1019
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.9928 - loss: 0.0215 - val_accuracy: 0.9708 - val_loss: 0.0988
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.9907 - loss: 0.0217 - val_accuracy: 0.9724 - val_loss: 0.1044
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.9954 - loss: 0.0162 - val_accuracy: 0.9647 - val_loss: 0.1176
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.9974 - loss: 0.0106 - val_accuracy: 0.9585 - val_loss: 

In [8]:
import cv2
import numpy as np

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)[0][0]
    if prediction > 0.5:
        print("🔴 Non-Mouth Image Detected")
    else:
        print("✅ Mouth Image Detected")

# Example usage
predict_image("/content/drive/MyDrive/test_notMouth_img.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
✅ Mouth Image Detected
